In [1]:
import pyspark


In [2]:
pyspark.__version__

'3.5.5'

In [3]:
from pyspark.sql import SparkSession


In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()


Picked up JAVA_TOOL_OPTIONS: -Djava.security.manager=allow
Picked up JAVA_TOOL_OPTIONS: -Djava.security.manager=allow
25/03/07 21:57:19 WARN Utils: Your hostname, Yes-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.18.79 instead (on interface en0)
25/03/07 21:57:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/07 21:57:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/07 21:57:31 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [5]:
df = spark.read \
    .option("header", "true") \
    .parquet('fhv_tripdata_2021-01.parquet')


df.show()

In [9]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropOff_datetime', TimestampNTZType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', IntegerType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [10]:
!wc -l fhv_tripdata_2021-01.parquet

   32645 fhv_tripdata_2021-01.parquet


In [11]:
import pandas as pd

In [16]:
!head -n 101 fhv_tripdata_2021-01.parquet > head.parquet

In [14]:
!wc -l head.parquest

     101 head.parquest


In [18]:
df_pandas = pd.read_parquet('fhv_tripdata_2021-01.parquet')

In [21]:
df_subset = df_pandas.head(101)

In [22]:
df_subset.to_parquet('head.parquet', index=False)


In [23]:
df_pandas_head = pd.read_parquet('head.parquet')

In [24]:
df_pandas_head.dtypes

dispatching_base_num              object
pickup_datetime           datetime64[ns]
dropOff_datetime          datetime64[ns]
PUlocationID                     float64
DOlocationID                     float64
SR_Flag                           object
Affiliated_base_number            object
dtype: object

In [30]:
df_pandas_head = df_pandas_head.astype(str)  # Convert all columns to string (or use specific dtypes)


In [32]:
spark.createDataFrame(df_pandas_head).schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', StringType(), True), StructField('DOlocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [33]:
from pyspark.sql import types

In [40]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropOff_datetime', types.TimestampType(), True), 
    types.StructField('PUlocationID', types.DoubleType(), True), 
    types.StructField('DOlocationID', types.DoubleType(), True), 
    types.StructField('SR_Flag', types.IntegerType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
    ]
)

In [41]:
df = spark.read \
    .option("header", "true") \
    .schema(schema)\
    .parquet('fhv_tripdata_2021-01.parquet')


In [42]:
df.head(10)

[Row(dispatching_base_num='B00009', pickup_datetime=datetime.datetime(2021, 1, 1, 8, 27), dropOff_datetime=datetime.datetime(2021, 1, 1, 8, 44), PUlocationID=None, DOlocationID=None, SR_Flag=None, Affiliated_base_number='B00009'),
 Row(dispatching_base_num='B00009', pickup_datetime=datetime.datetime(2021, 1, 1, 8, 50), dropOff_datetime=datetime.datetime(2021, 1, 1, 9, 7), PUlocationID=None, DOlocationID=None, SR_Flag=None, Affiliated_base_number='B00009'),
 Row(dispatching_base_num='B00013', pickup_datetime=datetime.datetime(2021, 1, 1, 8, 1), dropOff_datetime=datetime.datetime(2021, 1, 1, 9, 51), PUlocationID=None, DOlocationID=None, SR_Flag=None, Affiliated_base_number='B00013'),
 Row(dispatching_base_num='B00037', pickup_datetime=datetime.datetime(2021, 1, 1, 8, 13, 9), dropOff_datetime=datetime.datetime(2021, 1, 1, 8, 21, 26), PUlocationID=None, DOlocationID=72.0, SR_Flag=None, Affiliated_base_number='B00037'),
 Row(dispatching_base_num='B00037', pickup_datetime=datetime.datetime(2

In [43]:
df = df.repartition(24)

In [45]:
df.write.parquet('fhvhv/2021/01/')

25/03/07 22:26:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/07 22:26:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/03/07 22:26:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/07 22:26:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/07 22:26:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/07 22:26:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/03/07 22:26:49 WARN MemoryManager: Total allocation exceeds 95.0

In [46]:
df = spark.read.parquet('fhvhv/2021/01/')

In [47]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropOff_datetime', TimestampType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', IntegerType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [48]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: double (nullable = true)
 |-- DOlocationID: double (nullable = true)
 |-- SR_Flag: integer (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [51]:
df.select('pickup_datetime', 'dropoff_datetime', 'PUlocationID', 'DOlocationID').show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PUlocationID|DOlocationID|
+-------------------+-------------------+------------+------------+
|2021-01-29 16:58:23|2021-01-29 17:34:40|        NULL|        NULL|
|2021-01-05 21:45:42|2021-01-05 21:57:16|        NULL|        72.0|
|2021-01-15 08:35:51|2021-01-15 08:42:46|        NULL|       254.0|
|2021-01-04 12:41:36|2021-01-04 12:46:44|        NULL|       197.0|
|2021-01-08 21:11:33|2021-01-08 21:31:39|        NULL|        63.0|
|2021-01-05 04:31:52|2021-01-05 04:38:10|        NULL|        61.0|
|2021-01-11 23:57:45|2021-01-12 00:20:04|        NULL|       217.0|
|2021-01-16 22:25:27|2021-01-16 22:30:22|        NULL|       140.0|
|2021-01-26 13:11:11|2021-01-26 13:23:25|        NULL|        94.0|
|2021-01-26 16:56:12|2021-01-26 17:08:37|        78.0|       250.0|
|2021-01-29 18:27:42|2021-01-29 18:30:22|        NULL|        39.0|
|2021-01-24 05:15:00|2021-01-24 05:34:00|       